In [57]:
import glob
import re

import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

path = "../recruit-restaurant-visitor-forecasting/"

In [58]:
# 读取数据
data = {
    'tra': pd.read_csv(path + 'air_visit_data.csv'),
    'as': pd.read_csv(path + 'air_store_info.csv'),
    'hs': pd.read_csv(path + 'hpg_store_info.csv'),
    'ar': pd.read_csv(path + 'air_reserve.csv'),
    'hr': pd.read_csv(path + 'hpg_reserve.csv'),
    'id': pd.read_csv(path + 'store_id_relation.csv'),
    'tes': pd.read_csv(path + 'sample_submission.csv'),
    'hol': pd.read_csv(path + 'date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [3]:
data["tra"].head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [4]:
data["tra"].shape

(252108, 3)

In [5]:
data["as"].head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [6]:
data["as"].shape

(829, 5)

In [7]:
data["hs"].head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [8]:
data["hs"].shape

(4690, 5)

In [9]:
data["ar"].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [10]:
data["ar"].shape

(92378, 4)

In [11]:
data["hr"].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [12]:
data["hr"].shape

(2000320, 4)

In [13]:
data["id"].head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [14]:
data["id"].shape

(150, 2)

In [15]:
data["hr"].hpg_store_id.value_counts()

hpg_2afd5b187409eeb4    1155
hpg_011e799ba201ad2e     822
hpg_9b20c78a9b8179d9     778
hpg_527c60506b80ac72     740
hpg_3f9e56ac6f9435c7     729
                        ... 
hpg_d5ef62c90f8207c5       2
hpg_d6e76691d2337299       2
hpg_8128f352a6fe9f92       2
hpg_8887294b4d37c8b0       2
hpg_d6fe283967e2765d       1
Name: hpg_store_id, Length: 13325, dtype: int64

可以看出hpg_id是有重复项的，同样的hpg_id代表同一家餐馆

In [16]:
data["ar"].air_store_id.value_counts()

air_8093d0b565e9dbdf    2263
air_e55abd740f93ecc4    1903
air_0a74a5408a0b8642    1831
air_cf5ab75a0afb8af9    1758
air_6d65542aa43b598b    1436
                        ... 
air_5485912b44f976de       1
air_d1418d6fd6d634f2       1
air_35512c42db0868da       1
air_2bffb19a24d11729       1
air_859feab8e3c9f98d       1
Name: air_store_id, Length: 314, dtype: int64

In [17]:
data["id"].hpg_store_id.value_counts()

hpg_8afe014b4a994a79    1
hpg_49dbe1fd79450e55    1
hpg_cad1acd4374ceaa1    1
hpg_dd709beb78198f38    1
hpg_e87d00be48cd91c6    1
                       ..
hpg_2864fa16b03e3813    1
hpg_ab64d4038f396dc3    1
hpg_fa4eeada9ad7b59f    1
hpg_5c05683a8f326588    1
hpg_50378da9ffb9b6cd    1
Name: hpg_store_id, Length: 150, dtype: int64

hpg没重复项，150个相互对应的hpg_id和air_id

In [18]:
# 内连接，筛选出h系统中已有的餐馆id
data["hr"] = pd.merge(data["hr"], data["id"], how="inner", on=["hpg_store_id"])

In [19]:
print(len(data['ar']))
print(len(data['hr']))

92378
28183


In [20]:
data["hr"].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926


In [21]:
# 将reserve_datetime的年月日和小时拆开
print(min(data["hr"]["reserve_datetime"].apply(lambda x: x.split(" ")[0])))
print(min(data["ar"]["reserve_datetime"].apply(lambda x: x.split(" ")[0])))
print(max(data["hr"]["reserve_datetime"].apply(lambda x: x.split(" ")[0])))
print(max(data["ar"]["reserve_datetime"].apply(lambda x: x.split(" ")[0])))

2016-01-01
2016-01-01
2017-04-22
2017-04-22


+ hpg系统预定时间2016-01-01   2017-04-22
+ air系统预定时间2016-01-01   2017-04-22

In [22]:
print(min(data['hr']['visit_datetime'].apply(lambda x:x.split(' ')[0])))
print(min(data['ar']['visit_datetime'].apply(lambda x:x.split(' ')[0])))
print(max(data['hr']['visit_datetime'].apply(lambda x:x.split(' ')[0])))
print(max(data['ar']['visit_datetime'].apply(lambda x:x.split(' ')[0])))

2016-01-01
2016-01-01
2017-05-31
2017-05-31


+ hpg系统访问时间2016-01-01   2017-05-31
+ air系统访问时间2016-01-01   2017-05-31

In [23]:
for df in ["ar", "hr"]:
    # 转换时间格式,只留年月日
    data[df]["visit_datetime"] = pd.to_datetime(data[df]["visit_datetime"])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    # 预约时间和访问时间的差
    data[df]["reserve_datetime_diff"] = data[df].apply(lambda r: (r["visit_datetime"] - r["reserve_datetime"]).days, axis=1)
    # 删除reserve_datetime列，增加rs1，rv1， rs2，rv2
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    # 合并
    data[df] = pd.merge(tmp1, tmp2, how="inner", on=["air_store_id", "visit_date"])

In [24]:
print(data['ar'].head())
print(data['hr'].head())

           air_store_id  visit_date  rs1  rv1  rs2   rv2
0  air_00a91d42b08b08d9  2016-10-31    0    2  0.0   2.0
1  air_00a91d42b08b08d9  2016-12-05    4    9  4.0   9.0
2  air_00a91d42b08b08d9  2016-12-14    6   18  6.0  18.0
3  air_00a91d42b08b08d9  2016-12-17    6    2  6.0   2.0
4  air_00a91d42b08b08d9  2016-12-20    2    4  2.0   4.0
           air_store_id  visit_date  rs1  rv1  rs2  rv2
0  air_00a91d42b08b08d9  2016-01-14    3    2  3.0  2.0
1  air_00a91d42b08b08d9  2016-01-15    6    4  6.0  4.0
2  air_00a91d42b08b08d9  2016-01-16    3    2  3.0  2.0
3  air_00a91d42b08b08d9  2016-01-22    3    2  3.0  2.0
4  air_00a91d42b08b08d9  2016-01-29    6    5  6.0  5.0


In [25]:
print(min(data['hr']['visit_date']))
print(min(data['ar']['visit_date']))
print(max(data['hr']['visit_date']))
print(max(data['ar']['visit_date']))

2016-01-01
2016-01-01
2017-05-31
2017-05-31


In [26]:
# 提交数据
data["tes"].head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [27]:
# air_visit_data
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek # 星期几
data['tra']['year'] = data['tra']['visit_date'].dt.year     # 年
data['tra']['month'] = data['tra']['visit_date'].dt.month   # 月
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date # 年月日

# submission data
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [28]:
data['tra'].head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [29]:
data["tra"].air_store_id.value_counts()

air_5c817ef28f236bdf    477
air_d97dabf7aae60da5    476
air_36bcf77d3382d36e    476
air_a083834e7ffe187e    476
air_60a7057184ec7ec7    475
                       ... 
air_1c0b150f9e696a5f     51
air_a17f0778617c76e2     47
air_789103bf53b8096b     41
air_a9a380530c1e121f     40
air_900d755ebd2f7bbd     20
Name: air_store_id, Length: 829, dtype: int64

In [30]:
data["tes"].head()

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


In [31]:
data["tes"].shape

(32019, 7)

In [32]:
data["tes"].air_store_id.value_counts()

air_6b7678aae65d2d59    39
air_32b02ba5dc2027f4    39
air_add9a575623726c8    39
air_09fd1f5c58583141    39
air_646b93e336f0dded    39
                        ..
air_d5e0a20370c325c7    39
air_4b9085d0d46a6211    39
air_681f96e6a6595f82    39
air_f927b2da69a82341    39
air_950381108f839348    39
Name: air_store_id, Length: 821, dtype: int64

提交文件tes中每个air_store_id都有39个预测

In [33]:
unique_stores = data["tes"]["air_store_id"].unique()  # 去重 排列
stores = pd.concat([pd.DataFrame({"air_store_id":unique_stores, "dow":[i] * len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

In [34]:
stores.head()

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0


In [35]:
# stores.shape
stores.dow.value_counts()

3    821
6    821
2    821
5    821
1    821
4    821
0    821
Name: dow, dtype: int64

tra中air_store_id的数量是829， 而stores中air_store_id的数量是821

获得tra中air_store_id和周几对应的访问数量(相关性大)，增加最小最大均值中位数计数特征

In [36]:
tmp = data["tra"].groupby(["air_store_id", "dow"], as_index=False)["visitors"].min().rename(columns={"visitors":"min_visitors"})
stores = pd.merge(stores, tmp, how="left", on=["air_store_id", "dow"])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

In [37]:
stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0


In [38]:
# 看看是否存在缺失值
stores.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5747 entries, 0 to 5746
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   air_store_id        5747 non-null   object 
 1   dow                 5747 non-null   int64  
 2   min_visitors        5689 non-null   float64
 3   mean_visitors       5689 non-null   float64
 4   median_visitors     5689 non-null   float64
 5   max_visitors        5689 non-null   float64
 6   count_observations  5689 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 359.2+ KB


In [39]:
data["as"].head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [40]:
data["as"].air_genre_name.value_counts()

Izakaya                         197
Cafe/Sweets                     181
Dining bar                      108
Italian/French                  102
Bar/Cocktail                     79
Japanese food                    63
Other                            27
Yakiniku/Korean food             23
Western food                     16
Okonomiyaki/Monja/Teppanyaki     14
Creative cuisine                 13
Asian                             2
Karaoke/Party                     2
International cuisine             2
Name: air_genre_name, dtype: int64

In [41]:
data["hol"].head()

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [42]:
stores = pd.merge(stores, data["as"], how="left", on=["air_store_id"]) # 连接as中可匹配的air_store_id

# NEW FEATURES FROM Georgii Vyshnia
stores["air_genre_name"] = stores["air_genre_name"].map(lambda x: str(str(x).replace("/"," ")))
stores["air_area_name"] = stores["air_area_name"].map(lambda x: str(str(x).replace("-"," ")))
lbl = preprocessing.LabelEncoder() # LabelEncoder
for i in range(10):
    # genre_name划分10次进行LabelEncoder
    stores["air_genre_name" + str(i)] = lbl.fit_transform(stores["air_genre_name"].map(lambda x: str(str(x).split(" ")[i]) if len(str(x).split(" ")) > i else ""))
    stores['air_area_name' + str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(" ")[i]) if len(str(x).split(" "))> i else ""))
# 原版进行labelencoder
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores["air_area_name"] = lbl.fit_transform(stores["air_area_name"])

data["hol"]["visit_date"] = pd.to_datetime(data["hol"]["visit_date"])
data["hol"]["visit_date"] = data["hol"]["visit_date"].dt.date  # 获取年月日
data["hol"]["day_of_week"] = lbl.fit_transform(data["hol"]["day_of_week"]) # labelEncoder

# data["hol"]是visit_date的解释，因此以visit_date合并tra
train = pd.merge(data["tra"], data["hol"], how="left", on=["visit_date"])
test = pd.merge(data["tes"], data["hol"], how="left", on=["visit_date"])

# stores：从test获得的air_store_id和dow，关联许多相关的信息
train = pd.merge(train, stores, how="left", on=["air_store_id", "dow"])
test = pd.merge(test, stores, how="left", on=["air_store_id", "dow"])

In [43]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# 合并ar和hr
for df in ["ar","hr"]:
    train = pd.merge(train, data[df], how="left", on=["air_store_id", "visit_date"])
    test = pd.merge(test, data[df], how="left", on=["air_store_id", "visit_date"])
    
# 对应tes的id
train["id"] = train.apply(lambda r: "_".join([str(r["air_store_id"]), str(r["visit_date"])]), axis=1)

# 关联ar和hr的特征
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

# 获取重要性强的特征
col = [c for c in train if c not in ["id", "air_store_id", "visit_date", "visitors"]]

# 暂时填充-1，可以优化填充值
train = train.fillna(-1)
test = test.fillna(-1)

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252108 entries, 0 to 252107
Data columns (total 53 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   air_store_id               252108 non-null  object 
 1   visit_date                 252108 non-null  object 
 2   visitors                   252108 non-null  int64  
 3   dow                        252108 non-null  int64  
 4   year                       252108 non-null  int64  
 5   month                      252108 non-null  int64  
 6   day_of_week                252108 non-null  int32  
 7   holiday_flg                252108 non-null  int64  
 8   min_visitors               252108 non-null  float64
 9   mean_visitors              252108 non-null  float64
 10  median_visitors            252108 non-null  float64
 11  max_visitors               252108 non-null  float64
 12  count_observations         252108 non-null  float64
 13  air_genre_name             25

In [47]:
# 训练预测
def RMSE(y, pred):
    return metrics.mean_squared_error(y ,pred) ** 0.5

model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, 
                                            random_state=3, 
                                            n_estimators=200,
                                            subsample=0.8,
                                            max_depth=10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8,
                    colsample_bytree=0.8, max_depth=12)

# visitors用log处理
model1.fit(train[col], np.log1p(train["visitors"].values))
model2.fit(train[col], np.log1p(train["visitors"].values))
model3.fit(train[col], np.log1p(train["visitors"].values))

# oof
preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])

print('RMSE GradientBoostingRegressor: ', RMSE(np.log1p(train['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSE(np.log1p(train['visitors'].values), preds2))
print('RMSE XGBRegressor: ', RMSE(np.log1p(train['visitors'].values), preds3))

preds1 = model1.predict(test[col])
preds2 = model2.predict(test[col])
preds3 = model3.predict(test[col])

# 手动设置权重
test["visitors"] = 0.3 * preds1 + 0.3*preds2 + 0.4*preds3
test["visitors"] = np.expm1(test["visitors"]).clip(lower=0.)
sub1 = test[["id","visitors"]].copy()
del train; del data

RMSE GradientBoostingRegressor:  0.3587559860235605
RMSE KNeighborsRegressor:  0.42293607663931865
RMSE XGBRegressor:  0.2528237161888481


### weighted mean

In [86]:
# 读取数据
data = {
    'tra': pd.read_csv(path + 'air_visit_data.csv'),
    'as': pd.read_csv(path + 'air_store_info.csv'),
    'hs': pd.read_csv(path + 'hpg_store_info.csv'),
    'ar': pd.read_csv(path + 'air_reserve.csv'),
    'hr': pd.read_csv(path + 'hpg_reserve.csv'),
    'id': pd.read_csv(path + 'store_id_relation.csv'),
    'tes': pd.read_csv(path + 'sample_submission.csv'),
    'hol': pd.read_csv(path + 'date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [87]:
print('holidays at weekends are not special, right?')
wkend_holidays = data["hol"].apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
print(wkend_holidays)
data["hol"].loc[wkend_holidays, 'holiday_flg'] = 0

holidays at weekends are not special, right?
0      False
1       True
2       True
3      False
4      False
       ...  
512    False
513    False
514    False
515    False
516    False
Length: 517, dtype: bool


In [88]:
print('add decreasing weights from now')
#date_info['weight'] = ((date_info.index + 1) / len(date_info))       # LB 0.509
#date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 2  # LB 0.503
#date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 3  # LB 0.500
#date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 4  # LB 0.498
data["hol"]['weight'] = ((data["hol"].index + 1) / len(data["hol"])) ** 5  # LB 0.497

add decreasing weights from now


In [89]:
data["hol"].head()

,visit_date,day_of_week,holiday_flg,weight
0,2016-01-01,Friday,1,2.707368e-14
1,2016-01-02,Saturday,0,8.663577e-13
2,2016-01-03,Sunday,0,6.578904e-12
3,2016-01-04,Monday,0,2.772345e-11
4,2016-01-05,Tuesday,0,8.460525e-11


In [90]:
data["tra"].head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [91]:
# I've consumed all my submissions for day. I suggest to try weight 1/x, exp(-x).
# It seems to me that Recruit sales may be stochastic and depends only on recent means.

print('weighted mean visitors for each (air_store_id, day_of_week, holiday_flag) or (air_store_id, day_of_week)')
visit_data = data["tra"].merge(data["hol"], on=["visit_date"], how='left')
# visit_data.drop('visit_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

weighted mean visitors for each (air_store_id, day_of_week, holiday_flag) or (air_store_id, day_of_week)


F:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  import sys


In [92]:
visit_data.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,3.258097,Wednesday,0,1.005227e-08
1,air_ba937bf13d40fb24,2016-01-14,3.496508,Thursday,0,1.456087e-08
2,air_ba937bf13d40fb24,2016-01-15,3.401197,Friday,0,2.055908e-08
3,air_ba937bf13d40fb24,2016-01-16,3.135494,Saturday,0,2.838881e-08
4,air_ba937bf13d40fb24,2016-01-18,1.945910,Monday,0,5.115756e-08


In [93]:
wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
# print(visitors.head())
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.
visitors.head()

           air_store_id day_of_week  holiday_flg         0
0  air_00a91d42b08b08d9      Friday            0  3.583535
1  air_00a91d42b08b08d9      Monday            0  3.203625
2  air_00a91d42b08b08d9      Monday            1  3.091042
3  air_00a91d42b08b08d9    Saturday            0  2.524065
4  air_00a91d42b08b08d9      Sunday            0  1.098612


,air_store_id,day_of_week,holiday_flg,visitors
0,air_00a91d42b08b08d9,Friday,0,3.583535
1,air_00a91d42b08b08d9,Monday,0,3.203625
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.524065
4,air_00a91d42b08b08d9,Sunday,0,1.098612


In [94]:
data["tes"].head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [95]:
print('prepare to merge with date_info and visitors')
data["tes"]['air_store_id'] = data["tes"].id.map(lambda x: '_'.join(x.split('_')[:-1]))
data["tes"]['visit_date'] = data["tes"].id.map(lambda x: x.split('_')[2])
data["tes"].drop('visitors', axis=1, inplace=True)
data["tes"] = data["tes"].merge(data["hol"], on='visit_date', how='left')
data["tes"] = data["tes"].merge(visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')

prepare to merge with date_info and visitors


In [96]:
data["tes"].head()

,id,air_store_id,visit_date,day_of_week,holiday_flg,weight,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.682692,1.098612
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.689848,3.203625
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.697064,3.325868
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.704340,3.353439
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.711677,3.475056


In [97]:
# fill missings with (air_store_id, day_of_week)
missings = data["tes"].visitors.isnull()
data["tes"].loc[missings, 'visitors'] = data["tes"][missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), how='left')['visitors_y'].values

# fill missings with (air_store_id)
missings = data["tes"].visitors.isnull()
data["tes"].loc[missings, 'visitors'] = data["tes"][missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), on='air_store_id', how='left')['visitors_y'].values
    
data["tes"]['visitors'] = data["tes"].visitors.map(pd.np.expm1)

data["tes"][['id', 'visitors']].to_csv('dumb_result.csv', float_format='%.4f', index=None)
print("done")

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  # This is added back by InteractiveShellApp.init_path()


done
